In [8]:
import numpy as np
import pandas as pd
import nltk
from fuzzywuzzy import fuzz

In [13]:
##Function that takes in 3 lists, the list of strings you want to compare, the list of strings you're comparing to and a list with the original way the string from list 1 was written.  
def matchWithFuzzyNames(l1, l2, original, row_num):
    match = pd.DataFrame(columns = ['original company names', 'clean company name', 'company matches', 'fuzz ratio'])
    for i in l1:
        matches = []
        score = []
        score_partial = []
        for j in l2:
            #due to time constraints, we set the fuzz.ratio to 100, however future programming should adjust this value to some other quantity to see if we get better matches
            if (fuzz.ratio(i,j)) == 100:
                matches.append(j)
                score.append(fuzz.ratio(i,j))
                #the partial ratio will return the ratio raw score of 100  if the shorter string is found an any of substrings of the larger string, used this as a safety net and potentially as an additional step to 
                score_partial.append((fuzz.partial_ratio(i,j)))
        match = match.append({ 'clean company name': i,'company matches': matches, 'fuzz ratio': score, 'fuzz partial ratio': score_partial},ignore_index=True)
        
    match['original company names'] = original
    match['original row number'] = row_num
 
#Will output a pandas dataframe with 4 columns
    return match

In [11]:
#Isabels function; I added List at the end to differentiate between this and the top function
#I am using this original function to do DNA
def matchWithFuzzyNamesList(l1, l2, original):
    match = pd.DataFrame(columns = ['original company names', 'clean company name', 'company matches', 'fuzz ratio'])
    for i in l1:
        matches = []
        score = []
        score_partial = []
        for j in l2:
            if (fuzz.ratio(i,j)) == 100:
                matches.append(j)
                score.append(fuzz.ratio(i,j))
                score_partial.append((fuzz.partial_ratio(i,j)))
        match = match.append({ 'clean company name': i,'company matches': matches, 'fuzz ratio': score, 'fuzz partial ratio': score_partial},ignore_index=True)
        
    match['original company names'] = original
        

 


    return match

<h2>Replicating Isabel's code</h2>

In [3]:
fda = pd.read_csv("../data/working/fda_clean.csv", index_col = [0])

In [14]:
fda.columns

Index(['FDA Companies ', 'Company Clean'], dtype='object')

In [15]:
fda_clean = fda.dropna()

In [16]:
fda.head()

,FDA Companies,Company Clean
0,3D IMAGING DRUG,d imaging drug
1,3M,m
2,3M DRUG DELIVERY,m drug delivery
3,AAIPHARMA LLC,aaipharma
4,ABBOTT LABS,abbott


In [17]:
fda_org = fda_clean['FDA Companies '].reset_index()

In [18]:
fda_org

,index,FDA Companies
0,0,3D IMAGING DRUG
1,1,3M
2,2,3M DRUG DELIVERY
3,3,AAIPHARMA LLC
4,4,ABBOTT LABS
...,...,...
968,970,ZO SKIN HEALTH
969,971,ZYDUS HLTHCARE
970,972,ZYDUS PHARMS USA
971,973,ZYDUS PHARMS USA INC


In [19]:
#this variable will hold the original index positions of the fda companies from the original data
fda_row = fda_org['index']

In [43]:
fda_row

0        0
1        1
2        2
3        3
4        4
      ... 
968    970
969    971
970    972
971    973
972    974
Name: index, Length: 973, dtype: int64

In [21]:
fda_comp = fda_org['FDA Companies ']

In [22]:
#This variable will hold all the original companies
fda_comp

0           3D IMAGING DRUG
1                        3M
2          3M DRUG DELIVERY
3             AAIPHARMA LLC
4               ABBOTT LABS
               ...         
968          ZO SKIN HEALTH
969          ZYDUS HLTHCARE
970        ZYDUS PHARMS USA
971    ZYDUS PHARMS USA INC
972         ZYDUS WORLDWIDE
Name: FDA Companies , Length: 973, dtype: object

In [23]:
fda_list = fda_clean['Company Clean']

In [24]:
#This variable will hold the clean names
fda_list

0       d imaging drug
1                    m
2      m drug delivery
3            aaipharma
4               abbott
            ...       
970            zo skin
971              zydus
972              zydus
973              zydus
974    zydus worldwide
Name: Company Clean, Length: 973, dtype: object

In [25]:
matching = matchWithFuzzyNames(fda_list, fda_list, fda_comp, fda_row)

In [26]:
matching

,original company names,clean company name,company matches,fuzz ratio,fuzz partial ratio,original row number
0,3D IMAGING DRUG,d imaging drug,[d imaging drug],[100],[100],0
1,3M,m,[m],[100],[100],1
2,3M DRUG DELIVERY,m drug delivery,[m drug delivery],[100],[100],2
3,AAIPHARMA LLC,aaipharma,[aaipharma],[100],[100],3
4,ABBOTT LABS,abbott,[abbott],[100],[100],4
...,...,...,...,...,...,...
968,ZO SKIN HEALTH,zo skin,[zo skin],[100],[100],970
969,ZYDUS HLTHCARE,zydus,"[zydus, zydus, zydus]","[100, 100, 100]","[100, 100, 100]",971
970,ZYDUS PHARMS USA,zydus,"[zydus, zydus, zydus]","[100, 100, 100]","[100, 100, 100]",972
971,ZYDUS PHARMS USA INC,zydus,"[zydus, zydus, zydus]","[100, 100, 100]","[100, 100, 100]",973


In [27]:
#original row number is the row number from fda_clean before the dropna()s

In [28]:
matching['company matches'] = matching['company matches'].astype(str)
matching['original row number'] = matching['original row number'].astype(str)
matching['fuzz ratio'] = matching['fuzz ratio'].astype(str)

In [29]:
testing = matching.groupby("clean company name", as_index = False)

In [30]:
testing

In [52]:
#group by clean company name, append the original company names, row numbers and ratio value to new colum
unique_comp = matching.groupby('clean company name', as_index = False).agg({'original company names': ', '.join, 
                                                                            'original row number': ', '.join,
                                                                            'company matches': ', '.join,
                                                                           'fuzz ratio': ', '.join} )

In [53]:
unique_comp

,clean company name,original company names,original row number,company matches,fuzz ratio
0,aaipharma,AAIPHARMA LLC,3,['aaipharma'],[100]
1,abbott,ABBOTT LABS,4,['abbott'],[100]
2,abbvie,"ABBVIE, ABBVIE INC","5, 8","['abbvie', 'abbvie'], ['abbvie', 'abbvie']","[100, 100], [100, 100]"
3,abbvie endocrine,"ABBVIE ENDOCRINE, ABBVIE ENDOCRINE INC","6, 7","['abbvie endocrine', 'abbvie endocrine'], ['ab...","[100, 100], [100, 100]"
4,abhai,"ABHAI INC, ABHAI LLC","9, 10","['abhai', 'abhai'], ['abhai', 'abhai']","[100, 100], [100, 100]"
...,...,...,...,...,...
791,zambon,ZAMBON SPA,968,['zambon'],[100]
792,zevacor,ZEVACOR PHARMA INC,969,['zevacor'],[100]
793,zo skin,ZO SKIN HEALTH,970,['zo skin'],[100]
794,zydus,"ZYDUS HLTHCARE, ZYDUS PHARMS USA, ZYDUS PHARMS...","971, 972, 973","['zydus', 'zydus', 'zydus'], ['zydus', 'zydus'...","[100, 100, 100], [100, 100, 100], [100, 100, 100]"


In [ ]:
#CSVs I need
 

#fda x ndc
#dna x dna
#dna x ndc

In [54]:
unique_comp.to_csv("shinyDashboardBI/fdaxfda.csv")

In [38]:
og = pd.read_excel("../data/original/fda_companies.xlsx")

In [39]:
og.head()

,FDA Companies
0,3D IMAGING DRUG
1,3M
2,3M DRUG DELIVERY
3,AAIPHARMA LLC
4,ABBOTT LABS


<h3>Using Isabel's Method on NDC</h3>

In [9]:
ndc = pd.read_csv("../data/working/ndc_clean.csv", index_col = [0])

In [34]:
ndc_clean = ndc.dropna()

In [35]:
ndc.columns

Index(['original_company', 'originalRow', 'cleaned_name'], dtype='object')

In [36]:
ndc_org = ndc_clean['original_company'].reset_index()

In [37]:
#Since I deleted the first 25 rows from the original data (they were just numbers), the original row numbers for these companies start at 25
ndc_org

,row,original_company
0,0,SPIRONOLACTONE 2%
1,1,-L'Oreal USA Products Inc
2,2,.Cardinal Health
3,3,.Church & Dwight Canada Corp
4,4,{Preferred Pharmaeutials INC.
...,...,...
6712,7018,Zydus Pharmaceuticals USA Inc
6713,7019,Zydus Pharmaceuticals USA Inc.
6714,7020,Zydus Technologies Limited
6715,7021,ZYGONE


In [38]:
#Since I deleted the first 25 rows from the original data (they were just numbers), the original row numbers for these companies start at 25
ndc_row = ndc_org['row'] + 25

In [39]:
ndc_row

0    25
1    26
2    27
3    28
4    29
Name: row, dtype: int64

In [40]:
ndc_comp = ndc_org['original_company']

In [41]:
ndc_list = ndc_clean['cleaned_name']

In [42]:
matching = matchWithFuzzyNames(ndc_list, ndc_list, ndc_comp, ndc_row)

In [43]:
matching

,original company names,clean company name,company matches,fuzz ratio,fuzz partial ratio,original row number
0,SPIRONOLACTONE 2%,spironolactone,[spironolactone],[100],[100],25
1,-L'Oreal USA Products Inc,-loreal,[-loreal],[100],[100],26
2,.Cardinal Health,cardinal,"[cardinal, cardinal, cardinal, cardinal, cardi...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",27
3,.Church & Dwight Canada Corp,church&dwight canada,"[church&dwight canada, church&dwight canada, c...","[100, 100, 100]","[100, 100, 100]",28
4,{Preferred Pharmaeutials INC.,preferred pharmaeutials,[preferred pharmaeutials],[100],[100],29
...,...,...,...,...,...,...
6712,Zydus Pharmaceuticals USA Inc,zydus,"[zydus, zydus, zydus, zydus]","[100, 100, 100, 100]","[100, 100, 100, 100]",7043
6713,Zydus Pharmaceuticals USA Inc.,zydus,"[zydus, zydus, zydus, zydus]","[100, 100, 100, 100]","[100, 100, 100, 100]",7044
6714,Zydus Technologies Limited,zydus,"[zydus, zydus, zydus, zydus]","[100, 100, 100, 100]","[100, 100, 100, 100]",7045
6715,ZYGONE,zygone,[zygone],[100],[100],7046


In [44]:
matching['company matches'] = matching['company matches'].astype(str)
matching['original row number'] = matching['original row number'].astype(str)
matching['fuzz ratio'] = matching['fuzz ratio'].astype(str)
matching['original row number'] = matching['original row number'].astype(str) #adding 25 because 25 numbers were removed from the og dataset before cleaning, so original row ended up being 25 less than it was suppose to

In [45]:
matching

,original company names,clean company name,company matches,fuzz ratio,fuzz partial ratio,original row number
0,SPIRONOLACTONE 2%,spironolactone,['spironolactone'],[100],[100],25
1,-L'Oreal USA Products Inc,-loreal,['-loreal'],[100],[100],26
2,.Cardinal Health,cardinal,"['cardinal', 'cardinal', 'cardinal', 'cardinal...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",27
3,.Church & Dwight Canada Corp,church&dwight canada,"['church&dwight canada', 'church&dwight canada...","[100, 100, 100]","[100, 100, 100]",28
4,{Preferred Pharmaeutials INC.,preferred pharmaeutials,['preferred pharmaeutials'],[100],[100],29
...,...,...,...,...,...,...
6712,Zydus Pharmaceuticals USA Inc,zydus,"['zydus', 'zydus', 'zydus', 'zydus']","[100, 100, 100, 100]","[100, 100, 100, 100]",7043
6713,Zydus Pharmaceuticals USA Inc.,zydus,"['zydus', 'zydus', 'zydus', 'zydus']","[100, 100, 100, 100]","[100, 100, 100, 100]",7044
6714,Zydus Technologies Limited,zydus,"['zydus', 'zydus', 'zydus', 'zydus']","[100, 100, 100, 100]","[100, 100, 100, 100]",7045
6715,ZYGONE,zygone,['zygone'],[100],[100],7046


In [46]:
unique_comp = matching.groupby('clean company name', as_index = False).agg({'original company names': ', '.join, 
                                                                            'original row number': ', '.join,
                                                                            'company matches': ', '.join,
                                                                           'fuzz ratio': ', '.join} )

In [47]:
unique_comp.head(50)

,clean company name,original company names,original row number,company matches,fuzz ratio
0,& welding supply&welding,"B & J Welding Supply, LTD. dba B & J Welding S...",877,['& welding supply&welding'],[100]
1,-,65364-110,54,['-'],[100]
2,-eleven,"7-Eleven, 7-Eleven Inc, 7-Eleven Inc., 7-Eleve...","56, 57, 58, 59","['-eleven', '-eleven', '-eleven', '-eleven'], ...","[100, 100, 100, 100], [100, 100, 100, 100], [1..."
3,-loreal,-L'Oreal USA Products Inc,26,['-loreal'],[100]
4,a,"A19, LLC",79,['a'],[100]
5,a ox welding,A OX Welding Supply Company Inc.,68,['a ox welding'],[100]
6,a oxygen&dme,A + OXYGEN & D.M.E.,67,['a oxygen&dme'],[100]
7,a plus,A Plus Home Medical,69,['a plus'],[100]
8,a&healthcare equipment repair,A & D Healthcare Equipment Repair,65,['a&healthcare equipment repair'],[100]
9,a&mill&welding,A & E Mill & Welding Supply Company,66,['a&mill&welding'],[100]


In [48]:
og = pd.read_excel("../data/original/BI DSPG Company Datasets/NDC_Company_Dataset.xls")

In [81]:
unique_comp.to_csv("shinyDashboardBI/ndcxndc.csv")

In [51]:
#because I deleted the first 25 rows (the original row number was 25 less than it was suppose to be, so I added 25 to the og row number above to accurately reflect the actual row the company was in prior to any deletions)
og.iloc[25]


Row Labels     SPIRONOLACTONE 2% 
Name: 25, dtype: object

In [ ]:
#Double checking dna and fda

In [52]:
dna = pd.read_csv("../data/working/dna_clean.csv", index_col = [0])

In [55]:
dna.iloc[62283]

Unnamed: 0.1                   103022
Code                           WOEOSA
Description          Wockhardt Bio AG
cleaned_companies       wockhardt bio
Name: 62283, dtype: object

In [21]:
fdaXdna = pd.read_csv("../data/working/fda_dna_matching.csv", index_col = [0])

In [22]:
fdaXdna

,clean fda company name,original fda company,fda row,corporate family,clean dna row,clean dna company,original dna company,fuzz ratio
0,aaipharma,AAIPHARMA LLC,3,aaipharma,[3820],['aaipharma'],['AAIPharma Services Corp.'],[100]
1,abbott,ABBOTT LABS,4,abbott,"[260, 273, 275]","['abbott', 'abbott', 'abbott']","['Abbott AG', 'Abbott Oy', 'Abbott Srl']","[100, 100, 100]"
2,abbvie,"ABBVIE, ABBVIE INC","5, 8",abbvie,[488],['abbvie'],['AbbVie Inc.'],[100]
3,abraxis bioscience,ABRAXIS BIOSCIENCE,11,abraxis bioscience,[3198],['abraxis bioscience'],"['Abraxis BioScience, Inc.']",[100]
4,aci,ACI HEALTHCARE LTD,15,aci,"[699, 700]","['aci', 'aci']","['ACI International', 'ACI Limited']","[100, 100]"
...,...,...,...,...,...,...,...,...
234,wilshire,WILSHIRE PHARMS INC,948,wilshire,"[61913, 61915]","['wilshire', 'wilshire']","['Wilshire Bank', 'Wilshire Technologies Inc']","[100, 100]"
235,wockhardt,"WOCKHARDT, WOCKHARDT LTD, WOCKHARDT USA","951, 953, 954",wockhardt,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
236,wockhardt bio,WOCKHARDT BIO AG,952,wockhardt bio,[62283],['wockhardt bio'],['Wockhardt Bio AG'],[100]
237,wyeth,WYETH PHARMS INC,957,wyeth,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"


<h2>DNA Matching</h2>

In [3]:
dna = pd.read_csv("../data/working/dna_clean.csv", index_col = [0])

In [4]:
del dna['Unnamed: 0.1']
del dna['Code']

In [5]:
dna.dropna(inplace = True)

In [6]:
dna.reset_index(inplace = True)

In [7]:
dna

,index,Description,cleaned_companies
0,0,AA PLC,aa
1,1,"Emperial Americas, Inc.",emperial americas
2,2,"American Academy of Allergy, Asthma and Immuno...",academy allergy asthma immunology
3,3,Bird Studies Canada,bird studies canada
4,4,Aesculap AG & Co. KG,aesculap&co
...,...,...,...
63949,64000,Boost Brent Oil 3x Leverage Daily ETP,boost brent oil x leverage daily etp
63950,64001,Boost Brent Oil 3x Short Daily ETP,boost brent oil x short daily etp
63951,64002,Nuveen High Income November 2021 Target Term Fund,nuveen high income november target term fund
63952,64003,SPDR SSGA Gender Diversity Index ETF,spdr ssga gender diversity index etf


In [8]:
new_original_company = dna['Description']

In [9]:
new_original_company.tail()

63949                Boost Brent Oil 3x Leverage Daily ETP
63950                   Boost Brent Oil 3x Short Daily ETP
63951    Nuveen High Income November 2021 Target Term Fund
63952                 SPDR SSGA Gender Diversity Index ETF
63953               Health Insurance Institute of Slovenia
Name: Description, dtype: object

In [12]:
#Running Isabels code
dna_matching = matchWithFuzzyNamesList(dna['cleaned_companies'].tolist(), dna['cleaned_companies'].tolist(), new_original_company)

KeyboardInterrupt: 

<h3>FDA x DNA</h3>

In [65]:
x = pd.read_csv("../data/working/fda_dna_matching.csv", index_col = [0])

In [74]:
x.tail()

,clean fda company name,original fda company,fda row,corporate family,clean dna row,clean dna company,original dna company,fuzz ratio
234,wilshire,WILSHIRE PHARMS INC,948,wilshire,"[61913, 61915]","['wilshire', 'wilshire']","['Wilshire Bank', 'Wilshire Technologies Inc']","[100, 100]"
235,wockhardt,"WOCKHARDT, WOCKHARDT LTD, WOCKHARDT USA","951, 953, 954",wockhardt,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
236,wockhardt bio,WOCKHARDT BIO AG,952,wockhardt bio,[62283],['wockhardt bio'],['Wockhardt Bio AG'],[100]
237,wyeth,WYETH PHARMS INC,957,wyeth,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"
238,zambon,ZAMBON SPA,968,zambon,[63778],['zambon'],['Zambon Company SpA'],[100]


In [5]:
fda.iloc[952]

FDA Companies     WOCKHARDT BIO AG
Company Clean        wockhardt bio
Name: 952, dtype: object

In [87]:
dna.iloc[28069]

Unnamed: 0.1               46741
Code                      IDECPH
Description          Biogen Inc.
cleaned_companies         biogen
Name: 28069, dtype: object

In [12]:
ndc.iloc[319]

original_company    Altaire Pharmaceuticals Inc.
originalRow                                  344
cleaned_name                             altaire
Name: 319, dtype: object